# Accelerating End-to-End Data Science Workflows # 

## 07 - การแยก, แปลง, และโหลด (Extract, Transform, and Load) ##

**สารบัญ**

ในโน้ตบุ๊กนี้ เราจะเรียนรู้พื้นฐานของการแยก, แปลง, และโหลด โน้ตบุ๊กนี้ครอบคลุมส่วนต่างๆ ดังนี้:

1.  [การแยก, แปลง, และโหลด (ETL)](#Extract,-Transform,-and-Load-(ETL))
    * [การแยก (Extract)](#Extract)
    * [การแปลง (Transform)](#Transform)
    * [การโหลด (Load)](#Load)
2.  [บันทึกเป็นรูปแบบ Parquet (Save to Parquet Format)](#Save-to-Parquet-Format)
    * [การอ่านจาก Parquet (Reading from Parquet)](#Reading-from-Parquet)
3.  [การเร่งความเร็ว ETL สำหรับงานปลายน้ำ (Accelerated ETL for Downstream Tasks)](#Accelerated-ETL-for-Downstream-Tasks)

## การแยก, แปลง, และโหลด (ETL) ##

กรณีการใช้งานที่สำคัญ แต่อาจไม่ได้รับการยกย่องเท่าที่ควรของ RAPIDS คือการแยก, แปลง, และโหลด หรือเรียกสั้นๆ ว่า ETL ซึ่งเป็นกระบวนการรวมข้อมูลที่ใช้เพื่อรวมข้อมูลจากหลายแหล่งเข้าเป็นที่เก็บข้อมูลเดียวที่สอดคล้องกัน เป้าหมายหลักคือ:

* รวมข้อมูลจากหลายแหล่งให้เป็นรูปแบบเดียวที่สอดคล้องกัน
* ปรับปรุงคุณภาพข้อมูลผ่านการทำความสะอาดและการตรวจสอบ
* ช่วยให้การวิเคราะห์ข้อมูลและการรายงานมีประสิทธิภาพมากขึ้น
* สนับสนุนการตัดสินใจที่ขับเคลื่อนด้วยข้อมูล

### การแยก (Extract) ###

**การแยก** คือขั้นตอนแรกที่ข้อมูลจะถูกรวบรวมจากระบบแหล่งที่มาต่างๆ แหล่งที่มาเหล่านี้อาจรวมถึง:
* ไฟล์คงที่ (csv, json)
* SQL RDBMS
* เว็บเพจ
* API

**หมายเหตุ**: cuDF ไม่มีวิธีดึงข้อมูลธุรกรรมจากฐานข้อมูล SQL ภายนอกไปยัง GPU โดยตรง วิธีแก้ปัญหาคือการอ่านด้วย pandas และสร้าง cuDF dataframe ด้วย `cudf.from_pandas()`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%load_ext cudf.pandas
# DO NOT CHANGE THIS CELL
import pandas as pd
import time

In [ ]:
# DO NOT CHANGE THIS CELL
dtype_dict={
    'age': 'int8', 
    'sex': 'object', 
    'county': 'object', 
    'lat': 'float32', 
    'long': 'float32', 
    'name': 'object'
}
        
df=pd.read_csv('./data/uk_pop.csv', dtype=dtype_dict)
df.head()

เมื่อนำเข้าข้อมูล สิ่งสำคัญคือการรวมเฉพาะคอลัมน์ที่เกี่ยวข้องเพื่อลดภาระหน่วยความจำและการคำนวณ

ด้านล่างนี้ เราจะอ่านข้อมูลจุดศูนย์กลางมณฑล


In [ ]:
centroid_df=pd.read_csv('county_centroid.csv')
centroid_df.columns=['county', 'lat_county_center', 'long_county_center']
centroid_df.head()

In [ ]:
%%cudf.pandas.line_profile
combined_df=df.merge(centroid_df, on='county')


### การแปลง (Transform) ###

ในขั้นตอน **การแปลง** ข้อมูลที่ถูกดึงออกมาจะถูกทำความสะอาด ตรวจสอบความถูกต้อง และแปลงเป็นรูปแบบที่เหมาะสมสำหรับการวิเคราะห์

ด้านล่างนี้ เราจะเพิ่มคอลัมน์ใหม่ ซึ่งแสดงถึงระยะทางของแต่ละบุคคลจากจุดศูนย์กลางมณฑลของตน

In [ ]:
%%cudf.pandas.line_profile
c=['lat', 'long']
combined_df['R']=((combined_df[c] - combined_df.groupby('county')[c].transform('mean')) ** 2).sum(axis=1) ** 0.5

การใช้การ join เพื่อดึงค่า lookup สามารถทำได้เร็วกว่าการคำนวณค่าเหล่านั้นขึ้นมาใหม่ ไม่ใช่เรื่องแปลกที่จะเก็บสถิติของกลุ่มไว้เพื่อจุดประสงค์นี้

In [ ]:
%%cudf.pandas.line_profile

# read in centroid data
centroid_df=pd.read_csv('county_centroid.csv')

# merge 
combined_df=df.merge(centroid_df, on='county', suffixes=['', '_county_center'])

# calculate distance from county center
combined_df['R']=((combined_df['lat']-combined_df['lat_county_center'])**2+(combined_df['long']-combined_df['long_county_center'])**2)**0.5

ด้านล่างนี้ เราจะกรองข้อมูลเพื่อรวมเฉพาะผู้ใหญ่


In [ ]:
%%cudf.pandas.line_profile

senior_df_filter=combined_df['age'] >= 60
senior_df=combined_df.loc[senior_df_filter]

display(senior_df.head())

### การโหลด (Load) ###

ขั้นตอนสุดท้ายคือ **การโหลด** ซึ่งข้อมูลที่ผ่านการแปลงจะถูกโหลดเข้าสู่ระบบเป้าหมาย ระบบเป้าหมายอาจเป็นฐานข้อมูลหรือไฟล์ สิ่งสำคัญคือต้องเป็นระบบที่มีประสิทธิภาพสำหรับงานปลายน้ำ

In [ ]:
senior_df.head()

In [ ]:
# DO NOT CHANGE THIS CELL
senior_df.to_csv('senior_df.csv', index=False)

**หมายเหตุ**: หากงานปลายน้ำเกี่ยวข้องกับการสอบถามและวิเคราะห์ข้อมูลเพิ่มเติม รูปแบบไฟล์ CSV อาจไม่ใช่ตัวเลือกที่ดีที่สุด

<a name='s1-6'></a>
## บันทึกในรูปแบบ Parquet ##

หลังจากประมวลผลข้อมูลแล้ว เราจะคงข้อมูลนั้นไว้เพื่อใช้ในภายหลัง [Apache Parquet](https://parquet.apache.org/) เป็นรูปแบบไบนารีแบบคอลัมน์ และได้กลายเป็นมาตรฐานโดยพฤตินัยสำหรับการจัดเก็บข้อมูลตารางปริมาณมาก การแปลงเป็นรูปแบบไฟล์ Parquet มีความสำคัญ และโดยทั่วไปควรหลีกเลี่ยงไฟล์ CSV ในผลิตภัณฑ์ข้อมูล แม้ว่ารูปแบบไฟล์ CSV จะสะดวกและอ่านได้ง่าย แต่การนำเข้าไฟล์ CSV จำเป็นต้องอ่านและแยกวิเคราะห์เรคคอร์ดทั้งหมด ซึ่งอาจเป็นคอขวด ในความเป็นจริง นักพัฒนาจำนวนมากจะเริ่มการวิเคราะห์โดยการแปลงไฟล์ CSV เป็นรูปแบบไฟล์ Parquet ก่อน มีเหตุผลมากมายที่ต้องใช้รูปแบบ Parquet สำหรับการวิเคราะห์:

* ลักษณะแบบคอลัมน์ของไฟล์ Parquet ช่วยให้สามารถทำการตัดคอลัมน์ได้ ซึ่งมักจะทำให้ประสิทธิภาพการสืบค้นข้อมูลดีขึ้นอย่างมาก
* ใช้เมตาดาตาเพื่อจัดเก็บ Schema และรองรับประเภทข้อมูลขั้นสูงมากขึ้น เช่น Categorical, Datetimes และอื่นๆ ซึ่งหมายความว่าการนำเข้าข้อมูลจะไม่ต้องมีการอนุมาน Schema หรือการระบุ Schema ด้วยตนเอง
* รวบรวมเมตาดาตาที่เกี่ยวข้องกับสถิติระดับ Row-group สำหรับแต่ละคอลัมน์ ซึ่งช่วยให้สามารถกรองแบบ Predicate pushdown ซึ่งเป็นการสืบค้นแบบ Pushdown ที่ช่วยให้การคำนวณเกิดขึ้นที่ "เลเยอร์ฐานข้อมูล" แทนที่จะเป็น "เลเยอร์เอนจินการดำเนินการ" ในกรณีนี้ เลเยอร์ฐานข้อมูลคือไฟล์ Parquet ในระบบไฟล์ และเอนจินการดำเนินการคือ Dask
* รองรับตัวเลือกการบีบอัดที่ยืดหยุ่น ทำให้จัดเก็บได้กะทัดรัดและพกพาสะดวกกว่าฐานข้อมูล

เราจะใช้ `.to_parquet(path)`[[doc]](https://docs.dask.org/en/stable/generated/dask.dataframe.to_parquet.html#dask-dataframe-to-parquet) เพื่อเขียนลงในไฟล์ Parquet โดยค่าเริ่มต้น ไฟล์จะถูกสร้างขึ้นในไดเร็กทอรีเอาต์พุตที่ระบุโดยใช้หลักการ `part.0.parquet`, `part.1.parquet`, `part.2.parquet`, ... และอื่นๆ สำหรับแต่ละพาร์ติชันใน DataFrame สิ่งนี้สามารถเปลี่ยนแปลงได้โดยใช้พารามิเตอร์ `name_function` การส่งออกหลายไฟล์ช่วยให้ Dask สามารถเขียนไปยังหลายไฟล์พร้อมกัน ซึ่งเร็วกว่าการเขียนไปยังไฟล์เดียว

<p><img src='images/parquet.png' width=240></p>

เมื่อทำงานกับชุดข้อมูลขนาดใหญ่ การถอดรหัสและการเข้ารหัสมักจะเป็นงานที่ใช้ต้นทุนสูง ความท้าทายนี้มักจะซับซ้อนขึ้นเมื่อขนาดข้อมูลเพิ่มขึ้น รูปแบบที่พบบ่อยในวิทยาศาสตร์ข้อมูลคือการแบ่งชุดข้อมูลตามคอลัมน์ การแบ่งแถว หรือทั้งสองอย่าง การย้ายการดำเนินการกรองเหล่านี้ไปยังเฟสการอ่านของเวิร์กโฟลว์สามารถ: 1) ลดเวลา I/O และ 2) ลดปริมาณหน่วยความจำที่จำเป็น ซึ่งเป็นสิ่งสำคัญสำหรับ GPU ที่หน่วยความจำอาจเป็นปัจจัยจำกัด รูปแบบไฟล์ Parquet ช่วยให้การอ่านแบบกรองเป็นไปได้ผ่าน **การตัดคอลัมน์ (column pruning)** และ **การกรอง Predicate แบบอิงสถิติ (statistic-based predicate filtering)** เพื่อข้ามส่วนของข้อมูลที่ไม่เกี่ยวข้องกับปัญหา ด้านล่างนี้คือเคล็ดลับบางประการสำหรับการเขียนไฟล์ Parquet:

* เมื่อเขียนข้อมูล การเรียงลำดับข้อมูลตามคอลัมน์ที่คาดว่าจะมีการใช้ฟิลเตอร์มากที่สุด หรือคอลัมน์ที่มีคาร์ดินาลิตีสูงสุด สามารถนำไปสู่ประโยชน์ด้านประสิทธิภาพอย่างมาก เมตาดาตาที่คำนวณสำหรับแต่ละกลุ่มแถวจะช่วยให้สามารถใช้ฟิลเตอร์ Predicate pushdown ได้อย่างเต็มที่
* การเขียนรูปแบบ Parquet ซึ่งต้องมีการประมวลผลชุดข้อมูลทั้งหมดใหม่ อาจมีต้นทุนสูง รูปแบบนี้ทำงานได้ดีอย่างน่าทึ่งสำหรับการใช้งานที่เน้นการอ่าน และการจัดเก็บและดึงข้อมูลที่มีความหน่วงต่ำ
* พาร์ติชันใน Dask DataFrame สามารถเขียนไฟล์พร้อมกันได้ ดังนั้นไฟล์ Parquet หลายไฟล์จึงถูกเขียนพร้อมกัน

ด้านล่างนี้ เราจะเขียนข้อมูลในรูปแบบ Parquet หลังจากเรียงลำดับตามมณฑล

In [ ]:
# DO NOT CHANGE THIS CELL
senior_df=senior_df.sort_values('county')

senior_df.to_parquet('senior_df.parquet', index=False)

In [ ]:
# DO NOT CHANGE THIS CELL
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### การอ่านจาก Parquet ###

การสอบถามข้อมูลในรูปแบบ Parquet สามารถทำได้เร็วกว่ามาก โดยเฉพาะอย่างยิ่งเมื่อขนาดข้อมูลเพิ่มขึ้น

ด้านล่างนี้ เราจะอ่านจากทั้งรูปแบบ CSV และ Parquet เพื่อเปรียบเทียบ


In [ ]:
%load_ext cudf.pandas
import pandas as pd
import time

In [ ]:
%%cudf.pandas.line_profile

sel=[('county', '=', 'BLACKPOOL')]
parquet_df=pd.read_parquet('senior_df.parquet', columns=['age', 'sex', 'county', 'lat', 'long', 'name', 'R'], filters=sel)
parquet_df=parquet_df.loc[parquet_df['county']=='BLACKPOOL']

In [ ]:
parquet_df['county'].unique()

In [ ]:
%%cudf.pandas.line_profile

df=pd.read_csv('./senior_df.csv', usecols=['age', 'sex', 'county', 'lat', 'long', 'name', 'R'])
df=df.loc[df['county']=='BLACKPOOL']

In [ ]:
df['county'].unique()

## การเร่งความเร็ว ETL สำหรับงาน DownStream  ##

การเร่งกระบวนการ ETL เป็นสิ่งสำคัญสำหรับวิทยาศาสตร์ข้อมูล เนื่องจากให้ประโยชน์ดังต่อไปนี้:

* **ข้อมูลเชิงลึกที่ทันเวลา**: ETL ที่เร็วขึ้นช่วยให้การวิเคราะห์ข้อมูลเป็นปัจจุบันมากขึ้น ทำให้นักวิทยาศาสตร์ข้อมูลสามารถทำงานกับข้อมูลล่าสุดได้
* **เพิ่มประสิทธิภาพการทำงาน**: เวลาประมวลผลที่ลดลงหมายความว่านักวิทยาศาสตร์ข้อมูลสามารถใช้เวลาในการวิเคราะห์และพัฒนาโมเดลได้มากขึ้น แทนที่จะรอให้ข้อมูลพร้อม
* **การจัดการชุดข้อมูลขนาดใหญ่**: กระบวนการ ETL ที่เร่งความเร็วสามารถจัดการข้อมูลปริมาณมากได้อย่างมีประสิทธิภาพมากขึ้น
* **ประสิทธิภาพด้านต้นทุน**: ETL ที่เร่งความเร็วสามารถลดทรัพยากรการคำนวณและเวลา ซึ่งนำไปสู่ต้นทุนโครงสร้างพื้นฐานที่ต่ำลง

<p><img src='images/etl.png' width=720></p>








In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**เยี่ยมมาก!** ไปยัง [โน้ตบุ๊กถัดไป](1-08_cudf-polars.ipynb) กันเลย